In [18]:
import os
import numpy as np
import re
from collections import defaultdict, deque
import tqdm

In [20]:
#list_folders = 'ECMA_Instances_2018-2019'
list_folders = 'test'

file_to_data = {}
for file in tqdm.tqdm(os.listdir(list_folders)):
    data = {}
    with open(os.path.join(list_folders,file), "r") as f:
        lines = f.readlines()
        data['n'] = int(re.findall(r"[-+]?\d*\.\d+|\d+", lines[0])[0])
        data['s'] = int(re.findall(r"[-+]?\d*\.\d+|\d+", lines[1])[0])
        data['t'] = int(re.findall(r"[-+]?\d*\.\d+|\d+", lines[2])[0])
        data['S'] = int(re.findall(r"[-+]?\d*\.\d+|\d+", lines[3])[0])
        data['d1'] = int(re.findall(r"[-+]?\d*\.\d+|\d+", lines[4])[1])
        data['d2'] = int(re.findall(r"[-+]?\d*\.\d+|\d+", lines[5])[1])
        
        p = re.findall(r"[-+]?\d*\.\d+|\d+", lines[6])
        data['p'] = [int(nb) for nb in p]
        
        ph = re.findall(r"[-+]?\d*\.\d+|\d+", lines[7])
        data['ph'] = [int(nb) for nb in ph]
        
        d = -1*np.ones((data['n'], data['n']))
        D = -1*np.ones((data['n'], data['n']))
        for i in range(9,len(lines)):
            nbs = re.findall(r"[-+]?\d*\.\d+|\d+", lines[i])
            d[int(nbs[0])-1, int(nbs[1])-1] = float(nbs[2])
            D[int(nbs[0])-1, int(nbs[1])-1] = float(nbs[3])
        
        data['d'] = d
        data['D'] = D
        
        file_to_data[file] = data
        


100%|████████████████████████████████████████████| 4/4 [00:00<00:00, 19.69it/s]


In [32]:
class Graph:
    def __init__(self):
        self.nodes = set()
        self.edges = defaultdict(list)
        self.distances = {}

    def add_node(self, value):
        self.nodes.add(value)

    def add_edge(self, from_node, to_node, distance):
        self.edges[from_node].append(to_node)
        self.edges[to_node].append(from_node)
        self.distances[(from_node, to_node)] = distance


def dijkstra(graph, initial):
    visited = {initial: 0}
    path = {}
    max_dist = max(graph.distances.values())
    nodes = set(graph.nodes)

    while nodes: 
        min_node = None
        for node in nodes:
            if node in visited:
                if min_node is None:
                    min_node = node
                elif visited[node] < visited[min_node]:
                    min_node = node

        if min_node is None:
            break

        nodes.remove(min_node)
        current_weight = visited[min_node]

        for edge in graph.edges[min_node]:
            if (min_node, edge) in graph.distances.keys():
                dist = graph.distances[(min_node, edge)]
            else:
                dist = 10000*max_dist
            weight = current_weight + dist
            if edge not in visited or weight < visited[edge]:
                visited[edge] = weight
                path[edge] = min_node

    return visited, path

def shortest_path(graph, origin, destination):
    visited, paths = dijkstra(graph, origin)
    full_path = deque()
    _destination = paths[destination]

    while _destination != origin:
        full_path.appendleft(_destination)
        _destination = paths[_destination]

    full_path.appendleft(origin)
    full_path.append(destination)

    return(visited[destination], list(full_path))

def build_graph(data, power):
    graph = Graph()
    n = data['n']
    d = data['d']
    D = data['D']
    p = data['p']
    for i in range(n):
        graph.add_node(str(i))
        
    for i in range(n):
        for j in range(n):
            if d[i,j] >0:
                long = d[i,j]*(1+D[i,j])*((p[i] + p[j])**power)
                graph.add_edge(str(i), str(j), long)
    return(graph)

def compute_poids(data, path, verbose):
    p = data['p']
    ph = data['ph']
    poids_init = 0
    d_tot = 0
    for node in path:
        poids_init += p[int(node)]
        d_tot += ph[int(node)]
        
    poids_robuste = poids_init + min(data['d2'], d_tot)
    if verbose:
        print('poids init : ' + str(poids_init))
        print('poids robuste : ' + str(poids_robuste))
        print('limite : ' + str(data['S']))

        print('d_poids incertain total : ' + str(d_tot))
        print('d_poids limite : ' + str(data['d2']))
    return(poids_robuste)

def knapSack(W, wt, val, n): 
    K = [[0 for x in range(W+1)] for x in range(n+1)] 
  
    # Build table K[][] in bottom up manner 
    for i in range(n+1): 
        for w in range(W+1): 
            if i==0 or w==0: 
                K[i][w] = 0
            elif wt[i-1] <= w: 
                K[i][w] = max(val[i-1] + K[i-1][w-wt[i-1]],  K[i-1][w]) 
            else: 
                K[i][w] = K[i-1][w]
                
    rem_capacity = W
    items_to_take = []
    
    #Determine which items to keep and what their total value is       
    for k in range(n-1,-1,-1):
        if(K[k][rem_capacity]>0):
            items_to_take.append(k);
            rem_capacity=rem_capacity-wt[k];

    return(items_to_take, K[n][W])
    

def compute_long(data, path, verbose):
    d = data['d']
    D = data['D']
    
    D_tot = []
    long = []
    
    for i in range(len(path)-1):
        assert(d[int(path[i]),int(path[i+1])] > 0), 'Trajet non réalisable'
        long.append(d[int(path[i]),int(path[i+1])]*(1+D[int(path[i]),int(path[i+1])]))
        D_tot.append(int(100*D[int(path[i]),int(path[i+1])]))
    d1 = 100*data['d1']
    n = len(D_tot)
    
    items_to_take, value = knapSack(d1, D_tot, long, n)
    
    for i in range(len(path)-1):
        if i not in items_to_take:
            value += d[int(path[i]),int(path[i+1])]
        
    if verbose:
        print('long : ' + str(value))
        print('D_long incertain total : ' + str(np.sum(np.array(D_tot)[items_to_take])))
        print('D_long limite :' + str(d1))

def compute_robuste_shortest_path(data, power, verbose=False):
    graph = build_graph(data, power)
    if verbose:
        print("Graph built")
    visited, paths = dijkstra(graph, str(data['s']))
    length, path = shortest_path(graph, str(data['s']-1), str(data['t']-1))
    if verbose:
        print('length : ' + str(length))
        print('path : ', path)
    poids_robuste =  compute_poids(data, path, verbose)
    compute_long(data, path, verbose)
    
    if poids_robuste > data['S']:
        return(False)
    else:
        return(True)
    

In [33]:
data = file_to_data['20_USA-road-d.BAY.gr']
compute_robuste_shortest_path(data, 0, verbose=True)

Graph built
length : 17125.809999999998
path :  ['14', '3', '8', '19', '16']
poids init : 49
poids robuste : 54
limite : 66
d_poids incertain total : 14
d_poids limite : 5
long : 16970.78
D_long incertain total : 125
D_long limite :200


True

In [34]:
for file_name, data in tqdm.tqdm(file_to_data.items()):
    #print(file_name)
    feasible = False
    power = 0
    while not feasible:
        feasible = compute_robuste_shortest_path(data, power, verbose=False)
        power += 1
        assert(power < 50), file_name

100%|████████████████████████████████████████████| 4/4 [00:00<00:00,  6.56it/s]
